# ALL CODE TAKEN FROM OPENROUTESERVICES EXAMPLE

## please modify before using. 


## please read: 
https://openrouteservice.org/2187-2/
https://saralgyaan.com/posts/set-passwords-and-secret-keys-in-environment-variables-maclinuxwindows-python-quicktip/
https://openrouteservice.org/dev/#/api-docs/optimization/post

In [6]:
import folium
import pandas as pd
from openrouteservice import client

In [7]:
import os

# in .bashrc add export API_KEY="[api key here]"
# so do this:
# nano .bashrc
# export API_KEY="[api key here]"
# ctrl+X, Y

# load jupyter notebook with:
# env API_KEY=$API_KEY jupyter notebook

api_key = os.environ.get('API_KEY') # Provide your personal API key
# you need to make one on the openrouteservices website. 

print(api_key != None) # DO NOT PRINT YOUR API KEY, OR PUSH IT TO GITHUB!!!!

True


In [8]:
df = pd.read_csv("../data/raw/full_property_data.csv")

coords_cols = ["longitude", "latitude"]
houses_df = df[["index", *coords_cols]]
houses_df["location"] = houses_df[coords_cols].values.tolist()

houses_df.set_index("index", inplace=True)
houses_df.drop(coords_cols, axis=1, inplace=True)
min_houses = houses_df[0:3]
min_houses.head()

/tmp/ipykernel_17208/3610457353.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  houses_df["location"] = houses_df[coords_cols].values.tolist()
/tmp/ipykernel_17208/3610457353.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  houses_df.drop(coords_cols, axis=1, inplace=True)


,location
index,
https://www.domain.com.au/3502-14-16-the-esplanade-st-kilda-vic-3182-16002767,"[144.9746821, -37.8650177]"
https://www.domain.com.au/4203-35-spring-street-melbourne-vic-3000-15939303,"[144.9740049, -37.8141725]"
https://www.domain.com.au/901-902-85-market-street-south-melbourne-vic-3205-14089455,"[144.9569041, -37.8301164]"


In [9]:
# set up the houses dictionary 
houses = min_houses.to_dict('index')
houses

{'https://www.domain.com.au/3502-14-16-the-esplanade-st-kilda-vic-3182-16002767': {'location': [144.9746821,
   -37.8650177]},
 'https://www.domain.com.au/4203-35-spring-street-melbourne-vic-3000-15939303': {'location': [144.9740049,
   -37.8141725]},
 'https://www.domain.com.au/901-902-85-market-street-south-melbourne-vic-3205-14089455': {'location': [144.9569041,
   -37.8301164]}}

In [11]:
ors = client.Client(key=api_key)
# Set up folium map
map1 = folium.Map(location=[-37.86, 144.97], tiles="Stamen Terrain", zoom_start=10, preferCanvas=True)

# Request of isochrones with 15 minute on foot.
params_iso = {'profile': 'foot-walking',
              'range': [900],  # 900/60 = 15 minutes
              'attributes': ['total_pop']  # Get population count for isochrones
              }

for name, house in houses.items():
    print(name)
    params_iso['locations'] = [house['location']]  # Add apartment coords to request parameters
    house['iso'] = ors.isochrones(**params_iso)  # Perform isochrone request
    folium.features.GeoJson(house['iso']).add_to(map1)  # Add GeoJson to map

    folium.map.Marker(list(reversed(house['location'])),  # reverse coords due to weird folium lat/lon syntax
                      icon=folium.Icon(color='lightgray',
                                       icon_color='#cc0000',
                                       icon='home',
                                       prefix='fa',
                                       ),
                      popup=name,
                      ).add_to(map1)  # Add apartment locations to map

map1

https://www.domain.com.au/3502-14-16-the-esplanade-st-kilda-vic-3182-16002767
https://www.domain.com.au/4203-35-spring-street-melbourne-vic-3000-15939303
https://www.domain.com.au/901-902-85-market-street-south-melbourne-vic-3205-14089455


In [28]:
# Common request parameters
params_poi = {'request': 'pois',
              'sortby': 'distance',
             'limit': 3}

# POI categories according to
# https://giscience.github.io/openrouteservice/documentation/Places.html
categories_poi = {'bus_stop': [588],
                  'bus_station': [587],
                  'railway_station': [604],
                  'train_station': [610],
                  'tram_stop': [605]}

for name, house in houses.items():
    house['categories'] = dict()  # Store in pois dict for easier retrieval
    params_poi['geojson'] = house['iso']['features'][0]['geometry']
    print("\n{} apartment".format(name))

    for typ, category in categories_poi.items():
        params_poi['filter_category_ids'] = category
        house['categories'][typ] = dict()
        house['categories'][typ]['geojson'] = ors.places(**params_poi)['features']  # Actual POI request
        print(f"\t{typ}: {len(house['categories'][typ]['geojson'])}")


https://www.domain.com.au/3502-14-16-the-esplanade-st-kilda-vic-3182-16002767 apartment
	bus_stop: 0
	bus_station: 0
	railway_station: 0
	train_station: 0
	tram_stop: 2

https://www.domain.com.au/4203-35-spring-street-melbourne-vic-3000-15939303 apartment
	bus_stop: 0
	bus_station: 0
	railway_station: 2
	train_station: 2
	tram_stop: 2

https://www.domain.com.au/901-902-85-market-street-south-melbourne-vic-3205-14089455 apartment
	bus_stop: 0
	bus_station: 0
	railway_station: 0
	train_station: 0
	tram_stop: 2


In [34]:
# Set up common request parameters
params_route = {'profile': 'foot-walking',
                'format_out': 'geojson',
                'geometry': 'true',
                'format': 'geojson',
                'instructions': 'false',
                "radiuses":[400]
                }

# Set up dict for font-awesome
style_dict = {'bus_stop': 'child',
              'train_station': 'shopping-cart',
              'tram_stop': 'scissors',
              'railway_station': 'child',
              'bus_station': 'scissors'
              }

# Store all routes from all apartments to POIs
for house in houses.values():
    for cat, pois in house['categories'].items():
        pois['durations'] = []
        for poi in pois['geojson']:
            poi_coords = poi['geometry']['coordinates']

            # Perform actual request
            params_route['coordinates'] = [house['location'],
                                           poi_coords
                                           ]
            json_route = ors.directions(**params_route)

            folium.features.GeoJson(json_route).add_to(map1)
            folium.map.Marker(list(reversed(poi_coords)),
                              icon=folium.Icon(color='white',
                                               icon_color='#1a1aff',
                                               icon=style_dict[cat],
                                               prefix='fa'
                                               )
                              ).add_to(map1)

            poi_duration = json_route['features'][0]['properties']['summary']['duration']
            pois['durations'].append(poi_duration)  # Record durations of routes

map1

In [35]:
# Sum up the closest POIs to each apartment
for name, house in houses.items():
    house['shortest_sum'] = sum([min(cat['durations'] + [300]) for cat in house['categories'].values()])
    print(f"{name} apartment: {round(house['shortest_sum'] / 60, 1)} min"
          )

https://www.domain.com.au/3502-14-16-the-esplanade-st-kilda-vic-3182-16002767 apartment: 25.0 min
https://www.domain.com.au/4203-35-spring-street-melbourne-vic-3000-15939303 apartment: 24.9 min
https://www.domain.com.au/901-902-85-market-street-south-melbourne-vic-3205-14089455 apartment: 25.0 min
